In [1]:
#!/usr/bin/env python
import wuml


##	We generated a synthetic data for regression with 4 dimensions where
##	x1 x2 has positive influence
##	x3 has no influence
##	x4 has negative influence
##
##	x1 has normal distribution
##	x2 is exponential distribution but minus 2 so it could be negative
##	x3 is uniform but shouldn't matter
##	x4 is categorical distribution.


data = wuml.wData(xpath='../../data/shap_regress_example_mix_distributions.csv', batch_size=20, 
					label_type='continuous', label_column_name='label', row_id_with_label=0)

print(data)


         A       B       C    D
0   1.7641  0.8779  0.8379  3.0
1   0.4002 -0.8549  0.0961  0.0
2   0.9787 -1.5545  0.9765  3.0
3   2.2409 -1.4255  0.4687  1.0
4   1.8676 -0.8039  0.9768  3.0
5  -0.9773 -1.9379  0.6048  3.0
6   0.9501 -0.9011  0.7393  1.0
7  -0.1514 -0.8894  0.0392  3.0
8  -0.1032 -1.7638  0.2828  3.0
9   0.4106 -1.8620  0.1202  3.0
10  0.1440 -1.6210  0.2961  1.0
11  1.4543 -1.5479  0.1187  3.0
12  0.7610 -1.1556  0.3180  2.0
13  0.1217 -1.4227  0.4143  3.0
14  0.4439  2.4545  0.0641  3.0
15  0.3337 -1.8924  0.6925  1.0
16  1.4941 -1.7657  0.5666  3.0
17 -0.2052 -1.8241  0.2654  1.0
18  0.3131 -0.9413  0.5232  3.0
19 -0.8541 -1.7079  0.0939  3.0
20 -2.5530 -1.3721  0.5759  3.0
21  0.6536 -1.7197  0.9293  3.0
22  0.8644 -1.8269  0.3186  3.0
23 -0.7422 -1.8830  0.6674  3.0
24  2.2698 -0.9319  0.1318  3.0
25 -1.4544 -1.8513  0.7163  3.0
26  0.0458 -1.7811  0.2894  2.0
27 -0.1872 -1.5400  0.1832  3.0
28  1.5328 -0.2797  0.5865  0.0
29  1.4694 -1.8979  0.0201  1.0


In [3]:

#	Example 1
EXP = wuml.explainer(data, 	loss='mse',		# This will create a network for regression and explain instance wise 
						networkStructure=[(100,'relu'),(100,'relu'),(1,'none')], 
						max_epoch=150, learning_rate=0.001, print_network_training_status=False)

# Show the explanation results
explanation = EXP(data)	# outputs the weight importance
print('Notice that since x1 and x2 can be negative, we get both negative and positive influence.')
print('Note that dependening on if x1, x2 is + or -, they can have positive or negative influence.')
print(explanation)

Network Info:
	Learning rate: 0.001
	Max number of epochs: 150
	Cost Function: mse
	Train Loop Callback: None
	Cuda Available: True
	Network Structure
		Linear(in_features=4, out_features=100, bias=True) , relu
		Linear(in_features=100, out_features=100, bias=True) , relu
		Linear(in_features=100, out_features=1, bias=True) , none



  0%|          | 0/30 [00:00<?, ?it/s]

Notice that since x1 and x2 can be negative, we get both negative and positive influence.
Note that dependening on if x1, x2 is + or -, they can have positive or negative influence.
[[  4.9095   0.2256   0.4774 -39.5952]
 [  1.8174  -0.0599   0.1469   0.    ]
 [  3.6179  -2.2426   0.2079 -42.7322]
 [  6.9464  -0.2261   0.4853 -12.1998]
 [  6.0834  -0.7692   0.3511 -41.5538]
 [ -3.8562  -4.012   -0.0287 -40.7026]
 [  3.7189  -0.1424   0.5016 -12.8239]
 [ -0.4724  -1.7404  -0.0025 -41.3633]
 [ -0.3753  -3.7      0.058  -41.6958]
 [  1.5527  -3.3178   0.0358 -42.073 ]
 [  0.604   -1.3887   0.2164 -12.4459]
 [  5.0021  -2.0267   0.053  -42.269 ]
 [  3.0468  -0.9955   0.2606 -28.3394]
 [  0.4572  -2.785    0.1355 -42.2265]
 [  1.1114  -0.7499   0.0137 -38.1266]
 [  1.5251  -1.9252   0.4229 -12.3901]
 [  5.3333  -2.4209   0.2986 -42.4058]
 [ -0.8248  -1.7702   0.1447 -12.0405]
 [  1.1204  -1.4833   0.1596 -42.4081]
 [ -3.1415  -3.8208  -0.0009 -40.9017]
 [-12.099   -2.7361   0.004  -38.6157]

In [4]:
#	Example 2
Cdata = wuml.center_and_scale(data)
print(Cdata)

           A         B         C         D
0   1.221279  2.343443  1.396201  0.659380
1  -0.039432  0.438785 -1.145731 -2.307832
2   0.495300 -0.330201  1.871142  0.659380
3   1.662006 -0.188407  0.131060 -1.318761
4   1.316949  0.494843  1.872170  0.659380
5  -1.312715 -0.751626  0.597435  0.659380
6   0.468864  0.388003  1.058327 -1.318761
7  -0.549300  0.400863 -1.340711  0.659380
8  -0.504747 -0.560259 -0.505965  0.659380
9  -0.029819 -0.668199 -1.063147  0.659380
10 -0.276249 -0.403296 -0.460390 -1.318761
11  0.934918 -0.322946 -1.068288  0.659380
12  0.294071  0.108262 -0.385345 -0.329690
13 -0.296862 -0.185329 -0.055353  0.659380
14  0.000961  4.076410 -1.255386  0.659380
15 -0.100901 -0.701614  0.897957 -1.318761
16  0.971706 -0.562348  0.466535  0.659380
17 -0.599030 -0.626540 -0.565590 -1.318761
18 -0.119943  0.343816  0.317816  0.659380
19 -1.198836 -0.498815 -1.153270  0.659380
20 -2.769203 -0.129711  0.498403  0.659380
21  0.194796 -0.511785  1.709402  0.659380
22  0.38964

In [5]:

EXP2 = wuml.explainer(Cdata, 	loss='mse',		# This will create a network for regression and explain instance wise 
						networkStructure=[(100,'relu'),(100,'relu'),(1,'none')], 
						max_epoch=150, learning_rate=0.001, print_network_training_status=False)

# Show the explanation results
explanation = EXP2(Cdata)	# outputs the weight importance
print('Notice that centering does not help since x1, x2, x4 can be both positive and negative.')
print(explanation)

Network Info:
	Learning rate: 0.001
	Max number of epochs: 150
	Cost Function: mse
	Train Loop Callback: None
	Cuda Available: True
	Network Structure
		Linear(in_features=4, out_features=100, bias=True) , relu
		Linear(in_features=100, out_features=100, bias=True) , relu
		Linear(in_features=100, out_features=1, bias=True) , none



  0%|          | 0/30 [00:00<?, ?it/s]

Notice that centering does not help since x1, x2, x4 can be both positive and negative.
[[  2.327    3.4531  -3.1003 -11.2729]
 [ -0.1813   1.5166  -3.3708  25.5377]
 [  1.332   -2.1095  -5.619  -12.9005]
 [  2.055   -0.9709   0.0289  18.7207]
 [  2.4713   2.286   -6.3714 -12.1759]
 [ -6.5548  -5.6742   0.228  -12.7554]
 [  1.6442   1.523   -3.0959  17.0172]
 [ -2.3951   2.2248  -6.9118 -13.1317]
 [ -2.2352  -4.1308  -1.6648 -13.6486]
 [ -0.1293  -4.6656  -4.4842 -13.4511]
 [ -1.5017  -2.5384  -1.1223  19.0857]
 [  2.9818  -2.1778  -4.2731 -13.2063]
 [  1.0062   0.6587  -1.5069   5.973 ]
 [ -1.3458  -1.3361  -0.1087 -14.0472]
 [  0.0036   0.0471  -3.7405 -10.9342]
 [ -0.4974  -4.3762  -1.805   18.0107]
 [  2.3987  -4.1767  -0.1684 -14.1967]
 [ -3.577   -4.1442  -1.4783  18.9908]
 [ -0.4051   2.0561   0.1305 -14.1229]
 [ -6.0966  -3.4487  -5.2216 -12.7724]
 [-18.9661  -0.8554   0.0284 -11.3684]
 [  0.559   -3.526   -4.4659 -13.0334]
 [  1.3902  -4.5861  -0.8628 -14.0255]
 [ -5.089   -5.

In [6]:
#	Example 3
Udata = wuml.use_cdf_to_map_data_between_0_and_1(data, output_type_name='wData')
print(Udata)

           A         B         C         D
0   0.841284  0.948501  0.883430  0.666667
1   0.474158  0.735364  0.149834  0.033333
2   0.649704  0.434987  0.958997  0.666667
3   0.918966  0.497393  0.553681  0.166667
4   0.860806  0.751837  0.959182  0.666667
5   0.134657  0.256615  0.690228  0.666667
6   0.641483  0.719669  0.828004  0.166667
7   0.310210  0.723713  0.069160  0.666667
8   0.323500  0.334584  0.383016  0.666667
9   0.477395  0.289685  0.186922  0.666667
10  0.395401  0.402711  0.400929  0.166667
11  0.774416  0.438194  0.184661  0.666667
12  0.585549  0.620712  0.429830  0.300000
13  0.388701  0.498735  0.518470  0.666667
14  0.487765  0.983120  0.101709  0.666667
15  0.453498  0.276247  0.785815  0.166667
16  0.783690  0.333696  0.645114  0.666667
17  0.295711  0.306763  0.360159  0.166667
18  0.447118  0.705420  0.598501  0.666667
19  0.154188  0.360955  0.146420  0.666667
20  0.020466  0.522820  0.655987  0.666667
21  0.552803  0.355349  0.928893  0.666667
22  0.61645

In [7]:
EXP3 = wuml.explainer(Udata, 	loss='mse',		# This will create a network for regression and explain instance wise 
						networkStructure=[(600,'relu'),(600,'relu'),(600,'relu'),(1,'none')], 
						max_epoch=600, learning_rate=0.001, print_network_training_status=False)

# Show the regression results
Ŷ = EXP3.net(Udata, output_type='ndarray')
SR_train = wuml.summarize_regression_result(Udata.Y, Ŷ)
print(SR_train.true_vs_predict())

# Show the explanation results
# y = 5*x1 + x2 + x1*x2 - 8*x4 - 2*x4*x4 + 0.1*np.random.randn()
# Notice that x1 and x2 has positive influence, x4 has negative. 
explanation = EXP3(Udata)	# outputs the weight importance
print('If we map all the data into the range of [0,1], notice that x1, x2, x4 all have the correct attribution sign')
print('We noticed that it requires a larger network, and longer training time, but it gives different signed attribution.')
print(explanation)


Network Info:
	Learning rate: 0.001
	Max number of epochs: 600
	Cost Function: mse
	Train Loop Callback: None
	Cuda Available: True
	Network Structure
		Linear(in_features=4, out_features=600, bias=True) , relu
		Linear(in_features=600, out_features=600, bias=True) , relu
		Linear(in_features=600, out_features=600, bias=True) , relu
		Linear(in_features=600, out_features=1, bias=True) , none

Avg error: 0.0997

['y' 'ŷ']
[-30.88 -31.09]
[  0.9    0.77]
[-40.3  -40.32]
[ -3.22  -3.42]
[-35.01 -35.02]
[-47.01 -46.9 ]
[ -6.81  -6.97]
[-43.36 -43.41]
[-43.91 -43.98]
[-42.48 -42.55]
[-11.22 -11.4 ]
[-38.34 -38.44]
[-22.26 -22.32]
[-42.91 -42.99]
[-36.14 -36.26]
[-10.87 -10.99]
[-38.87 -39.03]
[-12.38 -12.43]
[-41.63 -41.7 ]
[-46.63 -46.72]
[-52.6  -52.63]
[-41.44 -41.62]
[-41.15 -41.16]
[-46.21 -46.27]
[-33.74 -33.83]
[-48.25 -48.46]
[-25.57 -25.57]
[-44.15 -44.25]
[  6.88   6.78]
[ -7.29  -7.44]



  0%|          | 0/30 [00:00<?, ?it/s]

If we map all the data into the range of [0,1], notice that x1, x2, x4 all have the correct attribution sign
We noticed that it requires a larger network, and longer training time, but it gives different signed attribution.
[[ 12.6422   2.4391  -1.517  -43.9453]
 [  4.5735  -0.5107  -0.2973  -2.2938]
 [  7.5708  -0.5416  -1.4997 -45.14  ]
 [  9.3192  -0.0029  -0.2084 -11.8181]
 [ 11.5955   0.9221  -1.9689 -44.8629]
 [  1.7972  -1.2354  -0.9405 -45.8143]
 [  8.0436  -0.2044  -1.6508 -12.4493]
 [  3.7041  -0.4159  -0.1272 -45.8659]
 [  3.9026  -0.9731  -0.2115 -45.9928]
 [  4.8119  -0.5981  -0.2166 -45.8408]
 [  3.7131  -1.171   -0.3468 -12.8875]
 [  7.8942  -0.3092  -0.3103 -45.0094]
 [  5.6276  -1.4397  -1.179  -24.6261]
 [  5.1458  -1.0522  -0.6241 -45.7527]
 [  7.1644   2.1792  -0.0783 -44.8175]
 [  4.6393  -0.7965  -1.4595 -12.6676]
 [  8.0804  -0.4811  -0.7491 -45.1722]
 [  2.7586  -1.1014  -0.3991 -12.9783]
 [  6.1289  -0.5663  -1.2573 -45.297 ]
 [  1.8272  -1.4073  -0.1827 -46.25